# This keyword
A tutorial on the use of the *this* keyword in JavaScript

## Top Level 
At the very top level the _this_ keyword refers to the **global** object in a Node REPL or the **window** object in the browser. In a Node module it refers to **module.exports**.

In [1]:
// Top level Node REPL
this === global;

true

In [ ]:
// Top level browser
this === window; //only works in a browser

In [2]:
// Top level Node module
this === module.exports;

false

## Plain Functions
In plain functions, _this_ behaves differently depending on `'strict mode'`.
When strict mode is off, _this_ refers to the global object.

In [3]:
function notInStrictMode(){
    return this === global;
}
notInStrictMode();

true

When strict mode is on, _this_ is undefined. This is the same
whether the strict mode is defined in the function or in the 
function's context.

In [4]:
function inStrictMode(){
    'use strict';
    return this === undefined;
}
inStrictMode();

true

### Explanation
This behaviour is enforced to help you avoid putting values into the global space by mistake. 
Consider the constructor function below. 

In [5]:
function Vehicle(make, model){
    this.make = make;
    this.model = model;
}


If this function is invoked without using the new keyword
what does _this_ refer to? Since _this_ in the Vehicle function
refers to the **global** object, the make and model become
global keys.

In [6]:
const veh = Vehicle("Toyota", "Corolla");
console.log('What does variable veh conatin: ', veh);
console.log('global.make: ', global.make);
console.log('global.model: ', global.model);

What does variable veh conatin:  undefined
global.make:  Toyota
global.model:  Corolla


The intention of course is to invoke this with the **new** keyword
in order to construct an object.

In [7]:
const vObj = new Vehicle("Toyota", "Corolla");
console.log(vObj);

Vehicle { make: 'Toyota', model: 'Corolla' }


This makes a good argument for always using **strict** mode.
Accidentally invoking the function will throw an appropriate error.

In [8]:
function StrictVehicle(make, model){
    'use strict';
    this.make = make;
    this.model = model;
}

const errV = StrictVehicle("Toyota", "Corolla");

TypeError: Cannot set property 'make' of undefined

## Methods
A method is a function that is a property of an object. **this** usually refers to
the object on which the method is invoked.

In [9]:
const fraction = {
    numerator:1,
    denominator:2,
    print() {
        console.log(`${this.numerator}/${this.denominator}`);
    }
}
fraction.print();

1/2


In this example _fraction_ is known as the **receiver** of the method call. It does not matter where the method was defined.

In [10]:
function print(){
    console.log(`${this.numerator}/${this.denominator}`);
}
fraction.print = print;
fraction.print();

1/2


In chained calls the receiver is the most immediate object in the chain.

In [11]:
const math = {
    foo: {
        fraction: fraction
    }
}

math.foo.fraction.print();

1/2


However, if the method is invoked in isolation, the **this** once again becomes the global because the function is called as a plain function.

In [12]:
const isolated_print = fraction.print;
isolated_print();

undefined/undefined


Such effects are most commonly experienced when passing functions as callbacks. 

In [13]:
const timer = setTimeout(fraction.print, 1000);

undefined/undefined


Wrapping the method in a function solves this problem. Since the function is called as a method the receiver remains the fraction object.

In [14]:
const timer2 = setTimeout(() => {fraction.print()});

1/2


## Call
Call is defined on the Object prototype so it is available on all functions. Call provides a means to invoke a function and specify the receiver without attaching the function to an object. 

In [15]:
function isPythagorean(){
    console.log(this.a, this.b, this.c);
    return (this.a**2 + this.b**2) === this.c**2;
}

const triangle1 = {
    a:2,
    b:3,
    c:4
}

const triangle2 = {
    a:3,
    b:4,
    c:5
}

console.log(isPythagorean.call(triangle1));
console.log(isPythagorean.call(triangle2));

2 3 4
false
3 4 5
true
